In [1]:
import client as clientlib
import transaction as translib
import block as blocklib
from database import *
from merkletree import *
import datetime

In [2]:
def keyToString(RSAkey):
    return RSAkey.split('-----')[2].replace('\n', '').strip()

def clientToString(i, client):
    s= ['[-----Client %d-----]' % i,
        'Public key: %s' % keyToString(client.public_key),
        'Identity: %s' % client.identity]
    return '\n'.join(s)

def transactionToString(i, trans):
    s= ['[-----Transaction %d-----]' % i]
    d= trans.to_dict()
    for x in d:
        s.append(str(x)+ ': ' + str(d[x]))
    return '\n'.join(s)


    

In [3]:
# -*- coding: utf-8 -*-

# Form implementation generated from reading ui file '/tmp/blockchainPR2953.ui'
#
# Created by: PyQt5 UI code generator 5.9.2
#
# WARNING! All changes made in this file will be lost!

from PyQt5 import QtCore, QtGui, QtWidgets

class Ui_MainWindow(object):
    
    ## Tab Client
    def on_clickNewClient(self):
        client= clientlib.newClient()
        d= client.to_dict()
        self.plainTextEditPrivateKey.document().setPlainText(keyToString(d['private_key']))
        self.plainTextEditPublicKey.document().setPlainText(keyToString(d['public_key']))
        self.plainTextEditIdentity.document().setPlainText(d['identity'])
  
    def updateListViewClient(self):
        self.clients= clientlib.getClients()
        model = QtGui.QStandardItemModel()
        i= 0
        for client in self.clients:
            i+= 1
            model.appendRow(QtGui.QStandardItem(clientToString(i, client)))
        self.listViewClient.setModel(model)
    
    def on_clickUpdateListClient(self):
        self.updateListViewClient()
    
    def initTabClient(self):
        self.tabClient = QtWidgets.QWidget()
        self.tabClient.setObjectName("tabClient")
        self.verticalLayout_3 = QtWidgets.QVBoxLayout(self.tabClient)
        self.verticalLayout_3.setObjectName("verticalLayout_3")
        self.tabCreateClient = QtWidgets.QTabWidget(self.tabClient)
        self.tabCreateClient.setObjectName("tabCreateClient")
        self.tabCreate = QtWidgets.QWidget()
        self.tabCreate.setObjectName("tabCreate")
        self.verticalLayout_4 = QtWidgets.QVBoxLayout(self.tabCreate)
        self.verticalLayout_4.setObjectName("verticalLayout_4")
        self.pushButtonCreateClient = QtWidgets.QPushButton(self.tabCreate)
        self.pushButtonCreateClient.setMaximumSize(QtCore.QSize(145, 16777215))
        self.pushButtonCreateClient.setObjectName("pushButtonCreateClient")
        self.verticalLayout_4.addWidget(self.pushButtonCreateClient)
        self.label = QtWidgets.QLabel(self.tabCreate)
        self.label.setObjectName("label")
        self.verticalLayout_4.addWidget(self.label)
        self.plainTextEditPrivateKey = QtWidgets.QPlainTextEdit(self.tabCreate)
        self.plainTextEditPrivateKey.setObjectName("plainTextEditPrivateKey")
        self.verticalLayout_4.addWidget(self.plainTextEditPrivateKey)
        self.label_2 = QtWidgets.QLabel(self.tabCreate)
        self.label_2.setObjectName("label_2")
        self.verticalLayout_4.addWidget(self.label_2)
        self.plainTextEditPublicKey = QtWidgets.QPlainTextEdit(self.tabCreate)
        self.plainTextEditPublicKey.setObjectName("plainTextEditPublicKey")
        self.verticalLayout_4.addWidget(self.plainTextEditPublicKey)
        self.label_25 = QtWidgets.QLabel(self.tabCreate)
        self.label_25.setObjectName("label_25")
        self.verticalLayout_4.addWidget(self.label_25)
        self.plainTextEditIdentity = QtWidgets.QPlainTextEdit(self.tabCreate)
        self.plainTextEditIdentity.setMaximumSize(QtCore.QSize(16777215, 120))
        self.plainTextEditIdentity.setObjectName("plainTextEditIdentity")
        self.verticalLayout_4.addWidget(self.plainTextEditIdentity)
        self.tabCreateClient.addTab(self.tabCreate, "")
        self.tabList = QtWidgets.QWidget()
        self.tabList.setObjectName("tabList")
        self.verticalLayout_5 = QtWidgets.QVBoxLayout(self.tabList)
        self.verticalLayout_5.setObjectName("verticalLayout_5")
        self.listViewClient = QtWidgets.QListView(self.tabList)
        self.listViewClient.setObjectName("listViewClient")
        self.verticalLayout_5.addWidget(self.listViewClient)
        self.pushButtonUpdateListClient = QtWidgets.QPushButton(self.tabList)
        self.pushButtonUpdateListClient.setMaximumSize(QtCore.QSize(100, 16777215))
        self.pushButtonUpdateListClient.setObjectName("pushButtonUpdateListClient")
        self.verticalLayout_5.addWidget(self.pushButtonUpdateListClient)
        self.tabCreateClient.addTab(self.tabList, "")
        self.verticalLayout_3.addWidget(self.tabCreateClient)
        self.tabWidget.addTab(self.tabClient, "")
        
        ###################
        self.pushButtonCreateClient.clicked.connect(self.on_clickNewClient)
        self.updateListViewClient() ###
        self.pushButtonUpdateListClient.clicked.connect(self.on_clickUpdateListClient) ##
        
        
    # Tab Transaction
    def selectionSender(self, i):
        if i == 0:
            identity= 'Genesis'
            self.sender= None
        else:
            identity= self.clients[i-1].identity
            self.sender= self.clients[i-1]
        self.plainTextEditSenderID.document().setPlainText(identity)
    
    def selectionRecipient(self, i):
        if i == 0:
            identity= 'Genesis'
            self.recipient= None
        else:
            identity= self.clients[i-1].identity
            self.recipient= self.clients[i-1]
        self.plainTextEditRecipientID.document().setPlainText(identity)
    
    def updateComboBoxClient(self):
        self.clients= clientlib.getClients()
        self.comboBoxSender.addItem('Genesis')
        self.comboBoxRecipient.addItem('None')
        for i in range(1, len(self.clients) + 1):
            self.comboBoxSender.addItem('Clients %d' % i)
            self.comboBoxRecipient.addItem('Clients %d' % i)
        self.selectionSender(0)
        self.selectionRecipient(0)
        
    def on_clickCreateTran(self):
        value= float(str.encode(self.plainTextEditValue.document().toPlainText()))
        tx= translib.createTransaction(self.sender, self.recipient.identity, value)
        self.plainTextEditMess.document().setPlainText(str(tx))
        
    def updateListViewUntransaction(self):
        self.untransactions= translib.getUntransactions()
        model = QtGui.QStandardItemModel()
        i= 0
        for utx in self.untransactions:
            i+= 1
            model.appendRow(QtGui.QStandardItem(transactionToString(i, utx)))
        self.listViewUntransaction.setModel(model)
    
    def on_clickUpdateListTransaction(self):
        self.updateListViewUntransaction()
        
    
    def initTabTransaction(self):
        self.tabTransaction = QtWidgets.QWidget()
        self.tabTransaction.setObjectName("tabTransaction")
        self.verticalLayout_2 = QtWidgets.QVBoxLayout(self.tabTransaction)
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.tabWidget_2 = QtWidgets.QTabWidget(self.tabTransaction)
        self.tabWidget_2.setObjectName("tabWidget_2")
        self.tabCreate1 = QtWidgets.QWidget()
        self.tabCreate1.setObjectName("tabCreate1")
        self.formLayout = QtWidgets.QFormLayout(self.tabCreate1)
        self.formLayout.setObjectName("formLayout")
        self.label_3 = QtWidgets.QLabel(self.tabCreate1)
        self.label_3.setObjectName("label_3")
        self.formLayout.setWidget(0, QtWidgets.QFormLayout.LabelRole, self.label_3)
        self.comboBoxSender = QtWidgets.QComboBox(self.tabCreate1)
        self.comboBoxSender.setMaximumSize(QtCore.QSize(145, 16777215))
        self.comboBoxSender.setObjectName("comboBoxSender")
        self.formLayout.setWidget(0, QtWidgets.QFormLayout.FieldRole, self.comboBoxSender)
        self.label_4 = QtWidgets.QLabel(self.tabCreate1)
        self.label_4.setObjectName("label_4")
        self.formLayout.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.label_4)
        self.plainTextEditSenderID = QtWidgets.QPlainTextEdit(self.tabCreate1)
        self.plainTextEditSenderID.setMaximumSize(QtCore.QSize(16777215, 50))
        self.plainTextEditSenderID.setObjectName("plainTextEditSenderID")
        self.formLayout.setWidget(1, QtWidgets.QFormLayout.FieldRole, self.plainTextEditSenderID)
        self.label_5 = QtWidgets.QLabel(self.tabCreate1)
        self.label_5.setMaximumSize(QtCore.QSize(145, 16777215))
        self.label_5.setObjectName("label_5")
        self.formLayout.setWidget(2, QtWidgets.QFormLayout.LabelRole, self.label_5)
        self.comboBoxRecipient = QtWidgets.QComboBox(self.tabCreate1)
        self.comboBoxRecipient.setMaximumSize(QtCore.QSize(145, 16777215))
        self.comboBoxRecipient.setObjectName("comboBoxRecipient")
        self.formLayout.setWidget(2, QtWidgets.QFormLayout.FieldRole, self.comboBoxRecipient)
        self.label_6 = QtWidgets.QLabel(self.tabCreate1)
        self.label_6.setObjectName("label_6")
        self.formLayout.setWidget(3, QtWidgets.QFormLayout.LabelRole, self.label_6)
        self.plainTextEditRecipientID = QtWidgets.QPlainTextEdit(self.tabCreate1)
        self.plainTextEditRecipientID.setMaximumSize(QtCore.QSize(16777215, 50))
        self.plainTextEditRecipientID.setObjectName("plainTextEditRecipientID")
        self.formLayout.setWidget(3, QtWidgets.QFormLayout.FieldRole, self.plainTextEditRecipientID)
        self.label_7 = QtWidgets.QLabel(self.tabCreate1)
        self.label_7.setObjectName("label_7")
        self.formLayout.setWidget(4, QtWidgets.QFormLayout.LabelRole, self.label_7)
        self.plainTextEditValue = QtWidgets.QPlainTextEdit(self.tabCreate1)
        self.plainTextEditValue.setMaximumSize(QtCore.QSize(200, 40))
        self.plainTextEditValue.setObjectName("plainTextEditValue")
        self.formLayout.setWidget(4, QtWidgets.QFormLayout.FieldRole, self.plainTextEditValue)
        self.pushButtonCreateTran = QtWidgets.QPushButton(self.tabCreate1)
        self.pushButtonCreateTran.setObjectName("pushButtonCreateTran")
        self.formLayout.setWidget(5, QtWidgets.QFormLayout.LabelRole, self.pushButtonCreateTran)
        self.plainTextEditMess = QtWidgets.QPlainTextEdit(self.tabCreate1)
        self.plainTextEditMess.setObjectName("plainTextEditMess")
        self.formLayout.setWidget(6, QtWidgets.QFormLayout.SpanningRole, self.plainTextEditMess)
        self.tabWidget_2.addTab(self.tabCreate1, "")
        self.tabPool = QtWidgets.QWidget()
        self.tabPool.setObjectName("tabPool")
        self.verticalLayout_6 = QtWidgets.QVBoxLayout(self.tabPool)
        self.verticalLayout_6.setObjectName("verticalLayout_6")
        self.listViewUntransaction = QtWidgets.QListView(self.tabPool)
        self.listViewUntransaction.setObjectName("listViewUntransaction")
        self.verticalLayout_6.addWidget(self.listViewUntransaction)
        self.pushButtonUpdateListTransaction = QtWidgets.QPushButton(self.tabPool)
        self.pushButtonUpdateListTransaction.setMaximumSize(QtCore.QSize(100, 16777215))
        self.pushButtonUpdateListTransaction.setObjectName("pushButtonUpdateListTransaction")
        self.verticalLayout_6.addWidget(self.pushButtonUpdateListTransaction)
        self.tabWidget_2.addTab(self.tabPool, "")
        self.verticalLayout_2.addWidget(self.tabWidget_2)
        self.tabWidget.addTab(self.tabTransaction, "")
        
        #####################
        self.updateComboBoxClient() ##
        self.comboBoxSender.currentIndexChanged.connect(self.selectionSender) ##
        self.comboBoxRecipient.currentIndexChanged.connect(self.selectionRecipient) ##
        self.pushButtonCreateTran.clicked.connect(self.on_clickCreateTran) ##
        self.plainTextEditMess.setFixedHeight(520)###
        
        self.updateListViewUntransaction()
        self.pushButtonUpdateListTransaction.clicked.connect(self.on_clickUpdateListTransaction)
    
    # Tab block
    def updateListViewUntransaction_BC(self):
        noTrans= int(str.encode(self.plainTextEditNoTran.document().toPlainText()))
        noTrans= min(noTrans, len(self.untransactions))
        self.listUntrans_createBlock= self.untransactions[: noTrans]
        model = QtGui.QStandardItemModel()
        i= 0
        for utx in self.listUntrans_createBlock:
            i+= 1
            model.appendRow(QtGui.QStandardItem(transactionToString(i, utx)))
        self.listViewUntransaction_BC.setModel(model)
        
        ##Merkle
        self.merkle= MerkleTree(self.listUntrans_createBlock)
        self.merkle.createTree()
        self.plainTextEditMerkleRoot.document().setPlainText(self.merkle.getRootLeaf())
        
    
    def on_clickUpdateUntrans(self):
        self.updateListViewUntransaction_BC()
        
    def on_clickPow(self):
        diff= int(str.encode(self.plainTextEditDifficulty.document().toPlainText()))
        time= str(datetime.datetime.now())
        self.previousBlock= blocklib.getLastBlock()
        previousHash = 'None' if self.previousBlock is None else self.previousBlock.hash
        self.block= blocklib.Block(0, 
                                   time, 
                                   [tx.hash for tx in self.listUntrans_createBlock],
                                   self.merkle.getRootLeaf(),
                                   previousHash)
        self.block.pow()
        self.block.make(self.block.nonce)
        self.plainTextEditNonce.document().setPlainText(str(self.block.nonce))
        self.plainTextEditTime.document().setPlainText(self.block.timestamp)
        self.plainTextEditBlockHash.document().setPlainText(self.block.hash)
        blocklib.save(self.block)
        txdb= TransactionDB()
        txdb.insert([tx.to_dict() for tx in self.listUntrans_createBlock])
        utxdb= UnTransactionDB()
        utxs= utxdb.read()
        utxs= utxs[len(self.listUntrans_createBlock):]
        utxdb.clear()
        utxdb.write(utxs)
        self.updateListViewUntransaction()
        self.updateListViewUntransaction_BC()
#         self.plainTextEditNonce.document().setPlainText('')
#         self.plainTextEditTime.document().setPlainText('')
#         self.plainTextEditBlockHash.document().setPlainText('')
    
    def updateBlockChain(self):
        self.blocks= blocklib.getBlocks()
        
    
    def viewBlock(self, id):
        try:
            block= self.blocks[id]
        except:
            return
        self.plainTextEditIndex.document().setPlainText(str(id+1))
        self.plainTextEditNoTran_BC.document().setPlainText(str(len(block.tx)))
        transactions= [translib.fromDict(TransactionDB().find(h)) for h in block.tx]
        model = QtGui.QStandardItemModel()
        i= 0
        for utx in transactions:
            i+= 1
            model.appendRow(QtGui.QStandardItem(transactionToString(i, utx)))
        self.listViewTransaction.setModel(model)
        
        self.plainTextEditMerkleRoot_BC.document().setPlainText(block.merkle)
    
    
    def initTabBlock(self):
        self.tabBlock = QtWidgets.QWidget()
        self.tabBlock.setObjectName("tabBlock")
        self.verticalLayout_7 = QtWidgets.QVBoxLayout(self.tabBlock)
        self.verticalLayout_7.setObjectName("verticalLayout_7")
        self.tabWidget_3 = QtWidgets.QTabWidget(self.tabBlock)
        self.tabWidget_3.setAutoFillBackground(False)
        self.tabWidget_3.setObjectName("tabWidget_3")
        self.tabCreateBlock = QtWidgets.QWidget()
        self.tabCreateBlock.setObjectName("tabCreateBlock")
        self.formLayout_2 = QtWidgets.QFormLayout(self.tabCreateBlock)
        self.formLayout_2.setObjectName("formLayout_2")
        self.label_8 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_8.setObjectName("label_8")
        self.formLayout_2.setWidget(0, QtWidgets.QFormLayout.LabelRole, self.label_8)
        self.plainTextEditNoTran = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditNoTran.setMaximumSize(QtCore.QSize(150, 30))
        self.plainTextEditNoTran.setObjectName("plainTextEditNoTran")
        self.formLayout_2.setWidget(0, QtWidgets.QFormLayout.FieldRole, self.plainTextEditNoTran)
        self.label_9 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_9.setObjectName("label_9")
        self.formLayout_2.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.label_9)
        self.listViewUntransaction_BC = QtWidgets.QListView(self.tabCreateBlock)
        self.listViewUntransaction_BC.setObjectName("listViewUntransaction_BC")
        self.formLayout_2.setWidget(1, QtWidgets.QFormLayout.FieldRole, self.listViewUntransaction_BC)
        self.label_10 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_10.setObjectName("label_10")
        self.formLayout_2.setWidget(4, QtWidgets.QFormLayout.LabelRole, self.label_10)
        self.plainTextEditPreviuosBlock = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditPreviuosBlock.setMaximumSize(QtCore.QSize(16777215, 50))
        self.plainTextEditPreviuosBlock.setObjectName("plainTextEditPreviuosBlock")
        self.formLayout_2.setWidget(4, QtWidgets.QFormLayout.FieldRole, self.plainTextEditPreviuosBlock)
        self.label_11 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_11.setObjectName("label_11")
        self.formLayout_2.setWidget(5, QtWidgets.QFormLayout.LabelRole, self.label_11)
        self.plainTextEditDifficulty = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditDifficulty.setMaximumSize(QtCore.QSize(140, 30))
        self.plainTextEditDifficulty.setObjectName("plainTextEditDifficulty")
        self.formLayout_2.setWidget(5, QtWidgets.QFormLayout.FieldRole, self.plainTextEditDifficulty)
        self.pushButtonUpdateUntran = QtWidgets.QPushButton(self.tabCreateBlock)
        self.pushButtonUpdateUntran.setMaximumSize(QtCore.QSize(200, 16777215))
        self.pushButtonUpdateUntran.setObjectName("pushButtonUpdateUntran")
        self.formLayout_2.setWidget(3, QtWidgets.QFormLayout.FieldRole, self.pushButtonUpdateUntran)
        self.label_12 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_12.setObjectName("label_12")
        self.formLayout_2.setWidget(7, QtWidgets.QFormLayout.LabelRole, self.label_12)
        self.plainTextEditNonce = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditNonce.setMaximumSize(QtCore.QSize(16777215, 30))
        self.plainTextEditNonce.setObjectName("plainTextEditNonce")
        self.formLayout_2.setWidget(7, QtWidgets.QFormLayout.FieldRole, self.plainTextEditNonce)
        self.label_13 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_13.setObjectName("label_13")
        self.formLayout_2.setWidget(2, QtWidgets.QFormLayout.LabelRole, self.label_13)
        self.plainTextEditMerkleRoot = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditMerkleRoot.setMaximumSize(QtCore.QSize(16777215, 60))
        self.plainTextEditMerkleRoot.setObjectName("plainTextEditMerkleRoot")
        self.formLayout_2.setWidget(2, QtWidgets.QFormLayout.FieldRole, self.plainTextEditMerkleRoot)
        self.label_14 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_14.setObjectName("label_14")
        self.formLayout_2.setWidget(8, QtWidgets.QFormLayout.LabelRole, self.label_14)
        self.plainTextEditTime = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditTime.setMaximumSize(QtCore.QSize(16777215, 30))
        self.plainTextEditTime.setObjectName("plainTextEditTime")
        self.formLayout_2.setWidget(8, QtWidgets.QFormLayout.FieldRole, self.plainTextEditTime)
        self.pushButtonPow = QtWidgets.QPushButton(self.tabCreateBlock)
        self.pushButtonPow.setMaximumSize(QtCore.QSize(16777215, 156))
        self.pushButtonPow.setObjectName("pushButtonPow")
        self.formLayout_2.setWidget(6, QtWidgets.QFormLayout.LabelRole, self.pushButtonPow)
        self.label_15 = QtWidgets.QLabel(self.tabCreateBlock)
        self.label_15.setObjectName("label_15")
        self.formLayout_2.setWidget(9, QtWidgets.QFormLayout.LabelRole, self.label_15)
        self.plainTextEditBlockHash = QtWidgets.QPlainTextEdit(self.tabCreateBlock)
        self.plainTextEditBlockHash.setMaximumSize(QtCore.QSize(16777215, 90))
        self.plainTextEditBlockHash.setObjectName("plainTextEditBlockHash")
        self.formLayout_2.setWidget(9, QtWidgets.QFormLayout.FieldRole, self.plainTextEditBlockHash)
        self.tabWidget_3.addTab(self.tabCreateBlock, "")
        self.tabBlockchain = QtWidgets.QWidget()
        self.tabBlockchain.setObjectName("tabBlockchain")
        self.formLayout_3 = QtWidgets.QFormLayout(self.tabBlockchain)
        self.formLayout_3.setObjectName("formLayout_3")
        self.label_16 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_16.setObjectName("label_16")
        self.formLayout_3.setWidget(2, QtWidgets.QFormLayout.LabelRole, self.label_16)
        self.plainTextEditIndex = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditIndex.setMaximumSize(QtCore.QSize(145, 30))
        self.plainTextEditIndex.setObjectName("plainTextEditIndex")
        self.formLayout_3.setWidget(2, QtWidgets.QFormLayout.FieldRole, self.plainTextEditIndex)
        self.label_17 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_17.setObjectName("label_17")
        self.formLayout_3.setWidget(3, QtWidgets.QFormLayout.LabelRole, self.label_17)
        self.plainTextEditNoTran_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditNoTran_BC.setMaximumSize(QtCore.QSize(145, 30))
        self.plainTextEditNoTran_BC.setObjectName("plainTextEditNoTran_BC")
        self.formLayout_3.setWidget(3, QtWidgets.QFormLayout.FieldRole, self.plainTextEditNoTran_BC)
        self.label_18 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_18.setObjectName("label_18")
        self.formLayout_3.setWidget(5, QtWidgets.QFormLayout.LabelRole, self.label_18)
        self.label_19 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_19.setObjectName("label_19")
        self.formLayout_3.setWidget(6, QtWidgets.QFormLayout.LabelRole, self.label_19)
        self.listViewTransaction = QtWidgets.QListView(self.tabBlockchain)
        self.listViewTransaction.setObjectName("listViewTransaction")
        self.formLayout_3.setWidget(5, QtWidgets.QFormLayout.FieldRole, self.listViewTransaction)
        self.plainTextEditMerkleRoot_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditMerkleRoot_BC.setMaximumSize(QtCore.QSize(16777215, 60))
        self.plainTextEditMerkleRoot_BC.setObjectName("plainTextEditMerkleRoot_BC")
        self.formLayout_3.setWidget(6, QtWidgets.QFormLayout.FieldRole, self.plainTextEditMerkleRoot_BC)
        self.label_20 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_20.setObjectName("label_20")
        self.formLayout_3.setWidget(7, QtWidgets.QFormLayout.LabelRole, self.label_20)
        self.plainTextEditDifficulty_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditDifficulty_BC.setMaximumSize(QtCore.QSize(145, 30))
        self.plainTextEditDifficulty_BC.setObjectName("plainTextEditDifficulty_BC")
        self.formLayout_3.setWidget(7, QtWidgets.QFormLayout.FieldRole, self.plainTextEditDifficulty_BC)
        self.label_21 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_21.setObjectName("label_21")
        self.formLayout_3.setWidget(8, QtWidgets.QFormLayout.LabelRole, self.label_21)
        self.plainTextEditNonce_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditNonce_BC.setMaximumSize(QtCore.QSize(16777215, 30))
        self.plainTextEditNonce_BC.setObjectName("plainTextEditNonce_BC")
        self.formLayout_3.setWidget(8, QtWidgets.QFormLayout.FieldRole, self.plainTextEditNonce_BC)
        self.label_22 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_22.setObjectName("label_22")
        self.formLayout_3.setWidget(9, QtWidgets.QFormLayout.LabelRole, self.label_22)
        self.plainTextEditTime_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditTime_BC.setMaximumSize(QtCore.QSize(16777215, 30))
        self.plainTextEditTime_BC.setObjectName("plainTextEditTime_BC")
        self.formLayout_3.setWidget(9, QtWidgets.QFormLayout.FieldRole, self.plainTextEditTime_BC)
        self.label_23 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_23.setObjectName("label_23")
        self.formLayout_3.setWidget(11, QtWidgets.QFormLayout.LabelRole, self.label_23)
        self.plainTextEditPreviuosBlock_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditPreviuosBlock_BC.setMaximumSize(QtCore.QSize(16777215, 145))
        self.plainTextEditPreviuosBlock_BC.setObjectName("plainTextEditPreviuosBlock_BC")
        self.formLayout_3.setWidget(11, QtWidgets.QFormLayout.FieldRole, self.plainTextEditPreviuosBlock_BC)
        self.label_24 = QtWidgets.QLabel(self.tabBlockchain)
        self.label_24.setObjectName("label_24")
        self.formLayout_3.setWidget(12, QtWidgets.QFormLayout.LabelRole, self.label_24)
        self.plainTextEditBlockHash_BC = QtWidgets.QPlainTextEdit(self.tabBlockchain)
        self.plainTextEditBlockHash_BC.setMaximumSize(QtCore.QSize(16777215, 145))
        self.plainTextEditBlockHash_BC.setObjectName("plainTextEditBlockHash_BC")
        self.formLayout_3.setWidget(12, QtWidgets.QFormLayout.FieldRole, self.plainTextEditBlockHash_BC)
        self.pushButtonPreviousBlock = QtWidgets.QPushButton(self.tabBlockchain)
        self.pushButtonPreviousBlock.setMaximumSize(QtCore.QSize(200, 16777215))
        self.pushButtonPreviousBlock.setObjectName("pushButtonPreviousBlock")
        self.formLayout_3.setWidget(14, QtWidgets.QFormLayout.LabelRole, self.pushButtonPreviousBlock)
        self.pushButtonNextBlock = QtWidgets.QPushButton(self.tabBlockchain)
        self.pushButtonNextBlock.setMaximumSize(QtCore.QSize(120, 16777215))
        self.pushButtonNextBlock.setObjectName("pushButtonNextBlock")
        self.formLayout_3.setWidget(14, QtWidgets.QFormLayout.FieldRole, self.pushButtonNextBlock)
        self.pushButtonUpdateBlockchain = QtWidgets.QPushButton(self.tabBlockchain)
        self.pushButtonUpdateBlockchain.setMaximumSize(QtCore.QSize(120, 16777215))
        self.pushButtonUpdateBlockchain.setLayoutDirection(QtCore.Qt.RightToLeft)
        self.pushButtonUpdateBlockchain.setObjectName("pushButtonUpdateBlockchain")
        self.formLayout_3.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.pushButtonUpdateBlockchain)
        self.tabWidget_3.addTab(self.tabBlockchain, "")
        self.verticalLayout_7.addWidget(self.tabWidget_3)
        self.tabWidget.addTab(self.tabBlock, "")
        #####################################
        self.plainTextEditNoTran.document().setPlainText('4')
        self.updateListViewUntransaction_BC()
        self.pushButtonUpdateUntran.clicked.connect(self.on_clickUpdateUntrans)
        self.previousBlock= blocklib.getLastBlock()
        previousHash = 'None' if self.previousBlock is None else self.previousBlock.hash
        self.plainTextEditPreviuosBlock.document().setPlainText(previousHash)
        self.plainTextEditDifficulty.document().setPlainText('3')
        self.pushButtonPow.clicked.connect(self.on_clickPow)
        self.updateBlockChain()
        self.viewBlock(0)
    
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(800, 924)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout.setObjectName("verticalLayout")
        self.tabWidget = QtWidgets.QTabWidget(self.centralwidget)
        self.tabWidget.setObjectName("tabWidget")
        
        self.initTabClient()
        self.initTabTransaction()
        self.initTabBlock()
        
        self.verticalLayout.addWidget(self.tabWidget)
        MainWindow.setCentralWidget(self.centralwidget)

        self.retranslateUi(MainWindow)
        self.tabWidget.setCurrentIndex(2)
        self.tabCreateClient.setCurrentIndex(1)
        self.tabWidget_2.setCurrentIndex(1)
        self.tabWidget_3.setCurrentIndex(1)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Blockchain"))
        self.pushButtonCreateClient.setText(_translate("MainWindow", "Creating client"))
        self.label.setText(_translate("MainWindow", "Private key: "))
        self.label_2.setText(_translate("MainWindow", "Public key:"))
        self.label_25.setText(_translate("MainWindow", "Identity:"))
        self.tabCreateClient.setTabText(self.tabCreateClient.indexOf(self.tabCreate), _translate("MainWindow", "Create client"))
        self.pushButtonUpdateListClient.setText(_translate("MainWindow", "Update"))
        self.tabCreateClient.setTabText(self.tabCreateClient.indexOf(self.tabList), _translate("MainWindow", "List of clients"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tabClient), _translate("MainWindow", "Client"))
        self.label_3.setText(_translate("MainWindow", "Sender: "))
        self.label_4.setText(_translate("MainWindow", "Identity sender: "))
        self.label_5.setText(_translate("MainWindow", "Recipient: "))
        self.label_6.setText(_translate("MainWindow", "Identity recipient: "))
        self.label_7.setText(_translate("MainWindow", "Value: "))
        self.pushButtonCreateTran.setText(_translate("MainWindow", "Creat transaction"))
        self.tabWidget_2.setTabText(self.tabWidget_2.indexOf(self.tabCreate1), _translate("MainWindow", "Create transaction"))
        self.pushButtonUpdateListTransaction.setText(_translate("MainWindow", "Update"))
        self.tabWidget_2.setTabText(self.tabWidget_2.indexOf(self.tabPool), _translate("MainWindow", "Pool"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tabTransaction), _translate("MainWindow", "Transaction"))
        self.label_8.setText(_translate("MainWindow", "No. transaction: "))
        self.label_9.setText(_translate("MainWindow", "List of untransaction: "))
        self.label_10.setText(_translate("MainWindow", "Previous block hash: "))
        self.label_11.setText(_translate("MainWindow", "Difficulty: "))
        self.pushButtonUpdateUntran.setText(_translate("MainWindow", "Update list transaction"))
        self.label_12.setText(_translate("MainWindow", "Nonce: "))
        self.label_13.setText(_translate("MainWindow", "Merkle root: "))
        self.label_14.setText(_translate("MainWindow", "Timestamp: "))
        self.pushButtonPow.setText(_translate("MainWindow", "PoW and Create block"))
        self.label_15.setText(_translate("MainWindow", "Block hash: "))
        self.tabWidget_3.setTabText(self.tabWidget_3.indexOf(self.tabCreateBlock), _translate("MainWindow", "Create block"))
        self.label_16.setText(_translate("MainWindow", "Index: "))
        self.label_17.setText(_translate("MainWindow", "No. transaction: "))
        self.label_18.setText(_translate("MainWindow", "List transaction:"))
        self.label_19.setText(_translate("MainWindow", "Merkle root: "))
        self.label_20.setText(_translate("MainWindow", "Difficulty: "))
        self.label_21.setText(_translate("MainWindow", "Nonce: "))
        self.label_22.setText(_translate("MainWindow", "Timestamp:"))
        self.label_23.setText(_translate("MainWindow", "Previous block: "))
        self.label_24.setText(_translate("MainWindow", "Block hash:"))
        self.pushButtonPreviousBlock.setText(_translate("MainWindow", "Previous block"))
        self.pushButtonNextBlock.setText(_translate("MainWindow", "Next block"))
        self.pushButtonUpdateBlockchain.setText(_translate("MainWindow", "Update"))
        self.tabWidget_3.setTabText(self.tabWidget_3.indexOf(self.tabBlockchain), _translate("MainWindow", "Blockchain"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tabBlock), _translate("MainWindow", "Block"))





In [4]:

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

KeyError: 'difficulty'